In [1]:
!pip install jsonpickle

In [2]:
#Import library yang dibutuhkan dalam proses pengambilan data
import tweepy,sys,jsonpickle
"""
=> Tweepy merupakan bagian package Python yang berfungsi untuk melakukan interaksi dengan API yang telah disediakan 
oleh twitter dengan mudah dalam pengambilan data. Tweepy disini nantinya akan digunakan untuk proses authentication API twitter.
=> jsonpickle digunakan untuk serialisasi dan deserialisasi objek Python kompleks ke dan dari JSON yang dimana dapat 
mengambil hampir semua objek Python dan mengubah objek tersebut menjadi JSON. Selain itu, dapat menyusun kembali objek tersebut
kembali ke Python. jsonpickle memungkinkan struktur data yang lebih kompleks untuk diserialkan ke JSON.
"""

"""
Agar python dan twitter terhubung dengan baik, terlebih dahulu harus mendapatkan authentication berupa akses token untuk
berinteraksi dengan Twitter API. Hal ini dilakukan agar bisa mengekstrak data dari Twitter.
"""
consumer_key = '*****' #di isi API key dari akun developer twitter
consumer_secret = '*****' #di isi API secret dari akun developer twitter
access_token = '*****'
access_secret = '*****'

qry='ekonomi psbb'#Kata kunci yang ingin dicari
maxTweets =  6000 #Jumlah maksimum tweet yang ingin kita ambil. Isi sembarang nilai sesuai kebutuhan
tweetsPerQry = 100  #Jumlah hasil yang diambil per Request. Jangan isi lebih dari 100, tidak diperbolehkan oleh Twitter
fName='ekonomi_psbb.json' #Nama file hasil Crawling untuk proses penyimpanan data

#Melakukan proses authentication API twitter
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
#Ada dua parameter selain authentikasi, wait_on_rate_limit, dan wait_on_rate_limit_notify 
#Digunakan untuk memanggil fungsi tidur otomatis di Tweepy ketika mencapai batas tingkat API Twitter.

#auth = tweepy.API(auth, proxy='https://your_proxy.server:port)'

if (not api):
    sys.exit('Autentikasi gagal, mohon cek "Consumer Key" & "Consumer Secret" Twitter anda')

sinceId, max_id, tweetCount = None, -1, 0

print("Mulai mengunduh maksimum {0} tweets".format(maxTweets))
with open(fName,'w') as f:
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=qry, tweet_mode="extended", count=tweetsPerQry, result_type="recent")
                else:
                    new_tweets = api.search(q=qry, tweet_mode="extended", count=tweetsPerQry, result_type="recent", 
                                            since_id=sinceId)
                    
            else:
                if (not sinceId):
                    new_tweets = api.search(q=qry, tweet_mode="extended", count=tweetsPerQry, result_type="recent", 
                                            max_id=str(max_id - 1))
                else:
                    new_tweets = api.search(q=qry, tweet_mode="extended", count=tweetsPerQry, result_type="recent", 
                                            max_id=str(max_id - 1),since_id=sinceId)
            
            """Penggunakan while loop untuk meminta semua tweet yang tersedia di qry kata kunci yang diinginkan. 
            Jika Statements pada Baris diatas digunakan untuk meminta data tweet, ketika max_id kurang dari 1, 
            program menjalankan query pencarian dari tweet terbaru dan kemudian menyimpan id terakhir ke max_id sehingga
            program dapat membuat permintaan lagi dari tweet terlama berdasarkan max_id.
            """      
            if not new_tweets:
                print('\nTidak ada lagi Tweet ditemukan dengan Query="{0}"'.format(qry));break
            for tweet in new_tweets:
                f.write(jsonpickle.encode(tweet._json,unpicklable=False)+'\n')
            tweetCount+=len(new_tweets)
            sys.stdout.write("\r");sys.stdout.write("Jumlah Tweets telah tersimpan: %.0f" %tweetCount);sys.stdout.flush()
            max_id=new_tweets[-1].id
        except tweepy.TweepError as e:
            print("some error : " + str(e));break # Aya error, keluar
print ('\nSelesai! {0} tweets tersimpan di "{1}"'.format(tweetCount,fName))

Mulai mengunduh maksimum 6000 tweets
Jumlah Tweets telah tersimpan: 139
Tidak ada lagi Tweet ditemukan dengan Query="ekonomi psbb"

Selesai! 139 tweets tersimpan di "ekonomi_psbb.json"


#### Convert JSON to CSV

In [ ]:
data = []
#Untuk membuka file JSON kemudian dibaca secara perbaris
for line in open('C:/Users/Niken Amelia/project_jupyter/TA/PSBB - EKONOMI/crawling/data/json baru (sdg diolah)/json (sdg diolah)/gabungan.json', 'r'):
    data.append(json.loads(line))

In [10]:
import pandas as pd
#Ubah bentuk JSON ke dalam dataframe, agar memudahkan perubahan format ke CSV
df = pd.json_normalize(data)

#Tampilkan dataframe yang sudah dibuat
df

In [12]:
#Seleksi atribut mana didalam dataframe yang nantinya ingin disimpan ke dalam bentuk CSV
saved_column = df[['full_text']]#you can also use df['column_name']

#Tampilkan atribut dari hasil seleksi sebelumnya
saved_column

In [14]:
#Simpan atribut tersebut ke dalam bentuk format CSV tanpa judul dan index
saved_column.to_csv('C:/Users/Niken Amelia/project_jupyter/TA/PSBB - EKONOMI/crawling/data/json baru (sdg diolah)/json (sdg diolah)/gabungan.csv', header=None, index=False)